In [1]:
from ProbRobNLP import FeatureExtraction, logic, dialogue
import copy

In [4]:
p = logic.Predicate("Vector3D", 3)
logic.Atom(p, [0, 0, 0])

Vector3D(0, 0, 0)

In [5]:
# (det a N) 
# (amod red N)
# (det the N)

# N: [x=det][N(x)]
# amod: A + N [A.ref = N.ref]
# det: N(det=D)



In [4]:
class Rule:
    pass


class DRSish:

    def __init__(self, ref, log, role=None, holes=None, fills=None):
        self.ref = ref
        self.log = log
        self.role = role
        self.holes = holes if holes is not None else {}
        self.fills = fills if fills is not None else {}

    def __repr__(self):
        return str(self.ref) + str(self.log) + ' h: ' + str(self.holes) + ' f: ' + str(self.fills)

    def __add__(self, other):
        ref = list(set(self.ref + other.ref))
        log = self.log + other.log
        self.holes.update(other.holes)
        self.fills.update(other.fills)
        gaps_to_fill = [h for h in self.fills.keys() if h in self.holes]

        # print('gaps to fill')
        # print(gaps_to_fill)
        
        drs = DRSish(ref, log, holes=self.holes, fills = self.fills)
        for g in gaps_to_fill:
            drs.replace_reference(drs.holes[g], drs.fills[g])
        # print(self.holes)
        return drs

    def get_predicates(self, reference):
        assert (reference in self.ref)

        for l in self.log:
            if reference in l.refs:
                yield l

    def get_first_predicates(self, reference):
        assert (reference in self.ref)

        for l in self.log:
            if reference == l.refs[0]:
                yield l

    def unify(self, other):

        new_ref = []
        log = copy.deepcopy(self.log)
        assert (len(other.ref) == 1)
        for ref in self.ref:

            if isinstance(ref, str) and '<' in ref:

                hole = ref.split('<')[1].strip('>')
                if hole == other.role:
                    new_ref.append(other.ref[0])
                    new_log = []
                    for formula in log:
                        if ref in formula.refs:
                            refs = [other.ref[0] if r == ref else r for r in formula.refs]
                            new_log.append(sem(formula.name, refs))
                        else:
                            new_log.append(formula)
                    log = new_log
            else:
                new_ref.append(ref)
        new_ref = [r for r in new_ref if isinstance(r, str)]
        return DRSish(new_ref, log + other.log, self.role)

    def join_on_reference(self, other):
        assert(len(self.ref) == len(other.ref))
        
        sref = self.ref
        oref = other.ref
        result = self + other
        
        for s, o in zip(sref, oref):
            result.replace_reference(o, s)
        return result
    
    def fill_gap(self, other, dep):
        if dep in self.holes:
            hole = self.holes.pop(dep)
            result = self + other
            result.replace_reference(hole, other.ref[0])
        else:
            self.fills[dep] = other.ref[0]
            result = self + other
        return result
        
    
    def replace_reference(self, hole, antecedent):
        new_ref = [r for r in self.ref if r != hole] + ([antecedent] if antecedent not in self.ref else [])
        log = copy.deepcopy(self.log)
        new_log = []
        for l in log:
            if hole in l.terms:
                refs = [r if r != hole else antecedent for r in l.terms]
                new_log.append(logic.Atom(l.pred, refs))
            else:
                new_log.append(l)
        self.ref = new_ref
        self.log = list(set(new_log))

    def to_PRS(self):
        order = self.order_prs()
        entities = {entity.name: entity for entity in self._to_PRS()}
        for name in order:
            yield entities[name]

    def remove_predicate(self, p):
        self.log = [l for l in self.log if p != l]

    def remove_reference(self, ref):
        self.ref = [r for r in self.ref if r != ref]

    def _to_PRS(self):
        types = {'tray': 'Tray', 'cup': 'Cup', 'bowl': 'Bowl', 'bucket': 'Bucket',
                 'chair': 'DiningChair', 'filled_cup': 'FilledCup',
                 'plate': 'Plate', 'table': 'Table', 'robot': 'Robot', 'camera': 'Camera', 'peg': 'HexagonalPegBase',
                 'gear': 'HexagonalGear', 'cube': 'Cube', 'toy_cube': 'ToyCube', 'cylinder': 'Cylinder',
                 'rope link': 'RopeLink',
                 'rope bucket': 'RopeBucket', 'conveyor belt': 'CircularConveyorBelt'}

        two_place = {'on': probrob.OnConstraint, 'in': probrob.OnConstraint}

        colour_dict = {'red': '"0.1"', 'blue': '"0.2"', 'green': '"0.3"', 'grey': '"0.4"', 'orange': '"0.5"',
                       'purple': '"0.7"', 'pink': '"0.6"', 'black': '"0"', 'white': '"1"'}

        # for l in self.log:
        #     if l.name in special_names:
        #         self.remove_predicate(l)
        #         self.replace_reference(l.refs[0], l.name)
        #         self.remove_reference(l.name)

        for ref in self.ref:

            preds = self.get_first_predicates(ref)

            constraints = []
            type_ = None
            for pred in preds:
                if pred.name in types:
                    type_ = types[pred.name]
                elif pred.name in two_place:
                    constraints.append(two_place[pred.name](
                        pred.refs[1] if pred.refs[1] not in special_names else special_names[pred.refs[1]]))
                elif pred.name in colour_dict:
                    constraints.append(probrob.PropConstraint('color', f'"{pred.name}"'))

            if type_ is not None:
                yield probrob.Entity(type_, ref, constraints)
            else:
                print(f'Warning: {ref} identified as type None')

    def order_prs(self):
        constraints = []
        for entity in self._to_PRS():
            for constraint in entity.constraints:
                try:
                    r = constraint.referent
                    constraints.append((r, entity.name))
                except AttributeError:
                    pass

        constraint_dict = {}
        for entity in self._to_PRS():
            c = [d for (d, i) in constraints if i == entity.name and d in self.ref]
            constraint_dict[entity.name] = c

        order = []
        i = 0
        while len(constraint_dict) > 0 and i < 20:
            print(constraint_dict)
            new_dict = {}
            for name, cons in constraint_dict.items():
                if cons == [] or all([x in order for x in cons]):
                    order.append(name)
                else:
                    new_dict[name] = cons
            constraint_dict = new_dict
            i += 1
        return order

    def draw(self):
        prs_lines = self.to_PRS()
        imp = probrob.Import('model', '*')
        scene = probrob.WorldModel(prs_lines, [imp])
        return scene

    def test_reference_matches(self, reference, constraints):

        preds = [c.name for c in self.get_first_predicates(reference)]
        # print('reference', reference, 'preds', preds, 'constraints', constraints)
        # print(all([c.name in preds for c in constraints]))
        return all([c.name in preds for c in constraints])


    def replace_specials(self):
        
        list_of_specials = {
            'centre': logic.Atom(logic.Predicate('Vector3D'), [0, 0, 0]),
            'left': logic.Atom(logic.Predicate('left', [])),
        }
    
    
class sem:
    def __init__(self, name, refs):
        self.name = name
        self.refs = refs

    def __repr__(self):
        return f'{str(self.name)}({", ".join(map(str, self.refs))})'

    def __eq__(self, other):
        return self.name == other.name and self.refs == other.refs

    def __hash__(self):
        # hash(custom_object)
        return hash((self.name, ' '.join(map(str, self.refs))))


     
    
class SExpression:
    
    def __init__(self, exp1, exp2, exp3):
        self.exp1 = exp1
        self.exp2 = exp2
        self.exp3 = exp3
        
    def __repr__(self):
        return f'({self.exp1} {str(self.exp2)} {self.exp3})'
    
    def __eq__(self, other):
        return self.exp1 == other.exp1 and self.exp2 == other.exp2 and self.exp3 == other.exp3  
    
    @classmethod
    def from_string(cls, s):
        s = s.strip('()')
        return SExpression(*s.split(' '))

class Sentence:

    def __init__(self, sentence):
        self.sentence = {i: w for i, w in enumerate(FeatureExtraction.get_words_and_tags(sentence))}

    def get_root(self):
        for word in self.sentence.values():
            if word['dep'] == 'ROOT':
                return word
        
    def get_head(self):
        for word in self.sentence.values():
            if word['id'] == word['head']:
                return word

    def get_children(self, id_):
        return [word for word in self.sentence.values() if word['head'] == id_ and word['id'] != id_]

    def get_leaves(self):
        heads = set([word['head'] for word in self.sentence.values()])
        return [word for word in self.sentence.values() if word['id'] not in heads]

    def __repr__(self):
        return '\n'.join([f"{i}: {w}" for i, w in self.sentence.items()])

    def __iter__(self):
        return self.sentence.items()
    
    def __getitem__(self, item):
        return self.sentence[item]
    
    def __setitem__(self, item, value):
        self.sentence[item] = value
    
    def items(self):
        return self.sentence.items()
    
    def __len__(self):
        return len(self.sentence)
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError as e:
            return default
    
    def join(self, head_id, child_id):
        head = self.sentence[head_id]
        child = self.sentence[child_id]

        head_words = [(i, w) for i, w in zip(head['word_positions'], head['word'].split())]
        child_words = [(i, w) for i, w in zip(child['word_positions'], child['word'].split())]
        
        new_words = sorted(head_words + child_words)
        
        ids, words = zip(*new_words)
        
        head['word'] = ' '.join(words)
        head['word_positions'] = ids

        self[head_id] = head

        self.sentence = {id_: word for id_, word in self.items()
                     if not (word['head'] == head['id'] and word['word'] in head['word']) or word['dep'] == 'ROOT'}
        
        for key, word in self.items():
            if word['head'] == child['id']:
                word['head'] = head['id']
    
def binarize(sentence):
    s = Sentence(sentence)
    
    return s

# s1 = SExpression('dobj', 'acquired', 'Pixar')
# print(s1)
# s2 = SExpression('nsubj', s1, 'Disney')
# print(s2)

# print(SExpression.from_string('(dobj acquired Pixar)'))

# assert(SExpression.from_string('(dobj acquired Pixar)') == s1)


dialogue.reset_reference()
dialogue.reset_variable()
dialogue.reset_event()

multiword_expressions = {
    ('the', 'centre'): (lambda: DRSish([logic.Atom(logic.Predicate('centre', 3), [0, 0, 0])], []), 0, 1),
    ('to', 'the', 'left', 'of'): (lambda x, y: DRSish([x, y], [logic.Atom(logic.Predicate('left_of', 2), [x,y])], holes={'dobj':x, 'pobj':y}), 2, 0),
    ('to', 'the', 'right', 'of'): (lambda x, y: DRSish([x, y], [logic.Atom(logic.Predicate('right_of', 2), [x,y])], holes={'dobj':x, 'pobj':y}), 2, 0)}


def combine_multword_expression(sentence, expression, start_location):
    words = [sentence[i] for i in range(start_location, start_location+len(expression))]
    heads = [w['head'] for w in words]
    leaves = [w for w in words if w['id'] not in heads]
    while leaves != []:
        next_one = leaves[0]
        sentence.join(next_one['head'], next_one['id'])
        words.remove(next_one)
        heads = [w['head'] for w in words]
        leaves = [w for w in words if w['id'] not in heads]
    return sentence, [words[0]]
        

def match_multiword_expression(sentence, multiword_expression, semantics):
    drs_lambda, num_variables, head_position = semantics
    for i in range(len(sentence)):
        if all([sentence.get(i+j, {}).get('word', '') == word for j, word in enumerate(multiword_expression)]):
            print("found match!", i, i+len(multiword_expression)-1)
            
            for j in range(i, i+len(multiword_expression)):
                if j != i+head_position:
                    sentence.join(sentence[j]['head'], j)
            drs = drs_lambda(*[dialogue.get_variable() for i in range(num_variables)])
            sentence[i+head_position]['sem'] = drs
    return sentence
                

s = Sentence('put a table on the centre')
print(match_multiword_expression(s, ('the', 'centre'), (lambda: DRSish([logic.Atom(logic.Predicate('centre', 3), [0, 0, 0])], []), 0, 1)))

s = Sentence('put a block to the left of the tray')
print(match_multiword_expression(s, ('to', 'the', 'left', 'of'), multiword_expressions[('to', 'the', 'left', 'of')]))


def substitution(sentence):
    
    for expression, semantics in multiword_expressions.items():
        sentence = match_multiword_expression(sentence, expression, semantics)
    
    for id_, word in sentence.items():
        if len(word['word'].split()) > 1:
            continue
        elif word['tag'] == 'DT':
            if word['word'] == 'a':
                word['sem'] = DRSish([logic.Constant(dialogue.get_reference())], [])
            elif word['word'] == 'the':
                word['sem'] = DRSish([logic.Variable(dialogue.get_variable())], [])
        elif word['tag'] in ['NN', 'JJ']:
            variable = logic.Variable(dialogue.get_variable())
            p = logic.Predicate(word['word'], 1)
            a = logic.Atom(p, [variable])
            word['sem'] = DRSish([variable], [a])
        
        elif word['tag'] == 'IN':
            x = logic.Variable(dialogue.get_variable())
            y = logic.Variable(dialogue.get_variable())
            p = logic.Predicate(word['word'], 2)
            a = logic.Atom(p, [x, y])
            word['sem'] = DRSish([x, y], [a], holes={'dobj':x, 'pobj':y})
        
        elif word['tag'] == 'VB':
            e = logic.Constant(dialogue.get_event())
            p = logic.Predicate(word['word'], 1)
            a = logic.Atom(p, [e])
            word['sem'] = DRSish([e], [a])
            
        elif word['tag'] == 'PRP':
            x = logic.Variable(dialogue.get_variable())
            word['sem'] = DRSish([x], [])
        
    return sentence

rules = {'amod': lambda head, child: head.join_on_reference(child),
         'det': lambda head, child: child.join_on_reference(head), 
         'pobj': lambda head, child: head.fill_gap(child, 'pobj'), 
         'prep': lambda head, child: head + child,
         'dobj': lambda head, child: head.fill_gap(child, 'dobj'),}


def compose(sentence, position):
    child = sentence.sentence[position]
    head_id = child['head']
    head = sentence.sentence[head_id]
    
    dep = child['dep']
    
    sem_child = child['sem']
    sem_head = head['sem']
    
#     print(head, child)
#     print(sem_head, sem_child)
    new_sem = rules[dep](sem_head, sem_child)
    
    sentence.sentence[head_id]['sem'] = new_sem
    sentence.join(head_id, child['id'])
    return sentence

def parse_sentence(s):
    s = Sentence(s)
    print(s)
    s = substitution(s)
    
    
    while ((leaves := s.get_leaves()) != []):
        l = leaves[0]
        # print('composing')
        # print(l, l['sem'].holes, l['sem'].fills)
        h = s.sentence[l['head']]
        # print(h, h['sem'].holes, h['sem'].fills)
        # print()
        compose(s, l['id'])
        # print(s)
        # print()
        # print()
    return s

# s = binarize('put a red block on the table')
# out = substitution(s)

# compose(s, 2)

# compose(s, 1)

# compose(s, 5)

# compose(s, 6)

# compose(s, 4)

# out = compose(s, 3)
# out



# s = 'put a red block on the table'
# parse_sentence(s)

# s = 'add a table to the centre'
# parse_sentence(s)

# s = 'a table'
# parse_sentence(s)

# s = 'put a tray on it'
# parse_sentence(s)

s = 'put a cube to the left of the green tray'
r = parse_sentence(s)
r

s = 'put the camera above the table'
r = parse_sentence(s)
r



found match! 4 5
0: {'id': 0, 'word': 'put', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'word_positions': [0]}
1: {'id': 1, 'word': 'a', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 2, 'ref': {}, 'word_positions': [1]}
2: {'id': 2, 'word': 'table', 'tag': 'NN', 'dep': 'dobj', 'pos': 'NOUN', 'head': 0, 'ref': {}, 'word_positions': [2]}
3: {'id': 3, 'word': 'on', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP', 'head': 0, 'ref': {}, 'word_positions': [3]}
5: {'id': 5, 'word': 'the centre', 'tag': 'NN', 'dep': 'pobj', 'pos': 'NOUN', 'head': 3, 'ref': {}, 'word_positions': (4, 5), 'sem': [centre(0, 0, 0)][] h: {} f: {}}
found match! 3 6
0: {'id': 0, 'word': 'put', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'word_positions': [0]}
1: {'id': 1, 'word': 'a', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 2, 'ref': {}, 'word_positions': [1]}
2: {'id': 2, 'word': 'block', 'tag': 'NN', 'dep': 'dobj', 'pos': 'NOUN', 'head': 0, 'ref': {}, 'word_positions':

0: {'id': 0, 'word': 'put the camera above the table', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'word_positions': (0, 1, 2, 3, 4, 5), 'sem': [X12, e1, X8][put(e1), above(X8, X12), table(X12), camera(X8)] h: {'dobj': X10} f: {'dobj': X8}}

In [11]:
r = 

'put a cube to the left of the green tray'

In [1]:
a = [1,2,3]
a.remove(1)
a

[2, 3]

In [106]:
s = 'add a cube to the left of the tray'
parse_sentence(s)

0: {'id': 0, 'word': 'add', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'word_positions': [0]}
1: {'id': 1, 'word': 'a', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 2, 'ref': {}, 'word_positions': [1]}
2: {'id': 2, 'word': 'cube', 'tag': 'NN', 'dep': 'dobj', 'pos': 'NOUN', 'head': 0, 'ref': {}, 'word_positions': [2]}
3: {'id': 3, 'word': 'to', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP', 'head': 0, 'ref': {}, 'word_positions': [3]}
4: {'id': 4, 'word': 'the', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 5, 'ref': {}, 'word_positions': [4]}
5: {'id': 5, 'word': 'left', 'tag': 'NN', 'dep': 'pobj', 'pos': 'NOUN', 'head': 3, 'ref': {}, 'word_positions': [5]}
6: {'id': 6, 'word': 'of', 'tag': 'IN', 'dep': 'prep', 'pos': 'ADP', 'head': 5, 'ref': {}, 'word_positions': [6]}
7: {'id': 7, 'word': 'the', 'tag': 'DT', 'dep': 'det', 'pos': 'DET', 'head': 8, 'ref': {}, 'word_positions': [7]}
8: {'id': 8, 'word': 'tray', 'tag': 'NN', 'dep': 'pobj', 'pos': 'NOUN', 'head': 6,

0: {'id': 0, 'word': 'add a cube to the left of the tray', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'word_positions': (0, 1, 2, 3, 4, 5, 6, 7, 8), 'sem': [e0, X1, X3, x0, X7][cube(x0), to(X1, X7), left(X3), tray(X7), add(e0), of(x0, X7)] r: None}

In [61]:
out

0: {'id': 0, 'word': 'put on the table', 'tag': 'VB', 'dep': 'ROOT', 'pos': 'VERB', 'head': 0, 'ref': {}, 'sem': [X4, e0, X2][put(e0), on(X2, X4), table(X4)] r: None}
3: {'id': 3, 'word': 'a red block', 'tag': 'NN', 'dep': 'dobj', 'pos': 'NOUN', 'head': 0, 'ref': {}, 'sem': [x0][red(x0), block(x0)] r: None}

In [62]:
red = out.sentence[2]['sem']
block = out.sentence[3]['sem']


KeyError: 2

In [23]:
block.join_on_reference(red)

AssertionError: 